#Growing Entourage Plots for ICYSM
###Flight and Rank Variant

Damon Crockett, Software Studies Initiative, Qualcomm Institute, UCSD

In [1]:
DIR = "/Users/damoncrockett/Desktop/cities_20k/yahoo/Clusters_for_Visualization/"

In [2]:
LABELS = DIR+"Hierarchical_Cluster_Labels/"

In [3]:
import os

In [4]:
# hacky way to get category names...

for root,dirs,files in os.walk(LABELS+"2/"):
    cats = [f[:-4] for f in files]

In [17]:
num = 25

In [3]:
cats = ['general_concepts_abstract_ideas']

##IMAGE DATA

In [4]:
BASE = "/Users/damoncrockett/Desktop/cities_20k/"

In [5]:
import glob
import pandas as pd

####ADD LOCAL PATHS to cluster label data

In [ ]:
# Once it's done once, it needn't be done again, although there's no harm in redoing it; it changes nothing

"""
for root,dirs,files in os.walk(LABELS):
    for file in files:
        f = root+"/"+file
        tmp = pd.read_csv(f)
        n = len(tmp.index)
        local_path = []
        for i in range(n):
            local_path.append(BASE+tmp.city.loc[i]+"_20k/"+tmp.image_id.loc[i]+".jpg")
        tmp['local_path'] = local_path
        tmp.to_csv(f,index=False)
"""

####Read in all the image features as a single df, subset to centroid columns

In [6]:
feature_matrix = pd.read_csv(BASE+"yahoo/all.csv")
EX_CENTROIDS = DIR+"Hierarchical_Clustered/2/cluster_characterization_activities.csv"
ex_centroids = pd.read_csv(EX_CENTROIDS)
del ex_centroids['Unnamed: 0']
cols = list(ex_centroids.columns)
cols.append('ad_id')
feature_matrix = feature_matrix[cols]
feature_matrix.rename(columns={"ad_id":"image_id"},inplace=True)

In [7]:
feature_matrix.dropna(inplace=True)

In [8]:
feature_matrix.reset_index(drop=True,inplace=True)

##PLOTTING LOOP

In [9]:
# ok this is a terrible way to do this, but I'm pressed for time...

In [10]:
from sklearn.manifold import TSNE
import numpy as np
from shapely.geometry import Point
from PIL import Image, ImageDraw

####City Colors

In [11]:
# upper right corner because that is where objects are least likely to be, according to Miriam's computations

bbox = [48,4,60,16]

In [12]:
colors = ['chartreuse','cyan','fuchsia','indigo','yellow']
cities = ['bangkok','berlin','moscow','sao','tokyo']

In [13]:
cats

['general_concepts_abstract_ideas']

In [14]:
cat = 'general_concepts_abstract_ideas'

In [15]:
factor = 12

####Loop

In [18]:
# centroid data
CENTROIDS = DIR+"Hierarchical_Clustered/25/cluster_characterization_"+cat+".csv"
centroids = pd.read_csv(CENTROIDS)
del centroids['Unnamed: 0']
        
# tsne on cluster centroids
tsne = TSNE(n_components=2)
X = centroids.as_matrix()
tsne.fit(X)
subspace = pd.DataFrame(tsne.fit_transform(X),columns=["x","y"])
        
# calculating euclidean distances from centroid
IMAGES = LABELS+str(num)+"/"+cat+".csv"
images = pd.read_csv(IMAGES)
#images = images.sample(n=8221)
images.reset_index(drop=True,inplace=True)
euclidean_distance = []
n = len(images)
for i in range(n):
    image_id_i = images.image_id.loc[i]
    image_point = feature_matrix[feature_matrix.image_id==image_id_i]
    del image_point['image_id']
    tmp = image_point.as_matrix()
            
    # note: df is zero-indexed but cluster labels are 1-indexed
    cluster_integer = int(images.cluster_id.loc[i]) - 1
    euclidean_distance_i = np.linalg.norm(tmp - centroids[centroids.index==cluster_integer].as_matrix())
    euclidean_distance.append(euclidean_distance_i)
        
images['euclidean_distance'] = euclidean_distance
        
# flight and rank
subspace.sort('x',inplace=True)
subspace['flight'] = np.repeat(range(1,6),5)
        
subspace.sort(['flight','y'],inplace=True)
subspace['rank'] = range(1,6) * 5
        
subspace['x_grid'] = subspace['flight'] * factor
subspace['y_grid'] = subspace['rank'] * factor
        
# turn centroid grid coords into shapely points
centroid_point = []
n = len(subspace.index)
for i in range(n):
    centroid_point.append(Point(subspace.x_grid.loc[i],subspace.y_grid.loc[i]))
    
subspace['centroid_point'] = centroid_point
        
# grid list, a list of every grid location
grid_side = 6 * factor
x,y = range(grid_side) * grid_side, np.repeat(range(grid_side),grid_side)
grid_list = pd.DataFrame(x,columns=['x'])
grid_list['y'] = y
        
# make into shapely points
point = []
n = len(grid_list.index)
for i in range(n):
    point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))

grid_list['point'] = point
        
# removoal of centroids from list of open grid locations
open_grid = list(grid_list.point)
centroids = list(subspace.centroid_point)
open_grid = [item for item in open_grid if item not in centroids]
        
# plotting...
thumb_side = 150
px_w = thumb_side * grid_side
px_h = thumb_side * grid_side
        
canvas = Image.new('RGB',(px_w,px_h),(119,119,119))
        
def plot():
    n = len(subspace.index)
    for i in range(n):
        centroid = subspace.centroid_point.loc[i]
        try:
            # again, a workaround for indexing difference
            candidates = images[images.cluster_id==i+1]
            candidates.sort("euclidean_distance",inplace=True)
            best = candidates.iloc[0]
            im = Image.open(best.local_path)
            im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
            closest_open = min(open_grid,key=lambda x: centroid.distance(x))
            x = int(closest_open.x) * thumb_side
            y = int(closest_open.y) * thumb_side
                    
            # adding colored ellipse...
            city = best.city
            idx = cities.index(city)
            color = colors[idx]
            draw = ImageDraw.Draw(im)
            draw.ellipse(bbox,fill=color)
                    
            canvas.paste(im,(x,y))
            idx = images[images.local_path==best.local_path].index
            images.drop(idx,inplace=True)
            open_grid.remove(closest_open)
                
        except:
            pass
        
iterations = list(images.cluster_id.value_counts())[0]
for i in range(iterations):
    plot()
        
canvas.save("/Users/damoncrockett/Desktop/ICWSM/tsne_FR/tmp/"+str(num)+"_"+cat+".png")

/Library/Python/2.7/site-packages/pandas/core/frame.py:2915: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  inplace=inplace, kind=kind, na_position=na_position)
